In [ ]:
#| default_exp chatbot_tab

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
#from jupyter_mentor.chatbot import ChatBot, ChatBotModel, ChatBotView
from jupyter_mentor.madlib import MadLibModel, MadLibView
from jupyter_mentor.llm import FileModel

In [ ]:
#|export
class ChatBotModel(FileModel):

    selected_index = traitlets.Int()
    bot_identity = traitlets.Unicode()
    human_identity = traitlets.Unicode()
    bot_template = traitlets.Unicode()

    def __init__(self, madlibs_list):
        super().__init__()

        self.names = []
        self.madlib_models = []
        for value in madlibs_list:
            model = MadLibModel(**value)
            self.names.append(model.name)
            traitlets.link((self, 'bot_identity'), (model, 'bot_identity'))
            traitlets.link((self, 'human_identity'), (model, 'human_identity'))
            traitlets.link((self, 'bot_template'), (model, 'bot_template'))
            self.madlib_models.append(model)

        self.human_template = "{input_text}"
        self.human_identity = "I am a student"
        self.bot_template = ""
        self.bot_identity = "Your are a tutor."
        
    def prompt(self):
        chat_prompt = ChatPromptTemplate.from_messages(
            [self.madlib_models[self.selected_index].bot_message_prompt, self.madlib_models[self.selected_index].human_message_prompt]
        )
        keys = self.madlib_models[self.selected_index].variables
        values = self.madlib_models[self.selected_index].values
        kwargs = {key:value for key, value in zip(keys,values)}
        prompt = chat_prompt.format_prompt( **kwargs )
        bot = self.llm.invoke(prompt).content
        user = "\n".join([desc + ' ' + val for desc, val in zip(self.madlib_models[self.selected_index].descriptions, self.madlib_models[self.selected_index].values)])
        return (user, bot)
        
    def prompt_with_kwargs(self, kwargs):
        ret = self.llm.invoke(self.chat_prompt.format_prompt(**kwargs))
        return ret.content

In [ ]:
madlibs_list = [
    { 'name': 'Open Prompt',
     'template': '{input_text}',
        'variables': ['input_text'],
        'descriptions': [''],
        'placeholders': ['Message AI chatbot...']
    }, {'name': 'Metaphor',
        'template': "I'm having trouble understanding {concept}. Please explain it as a metaphor",
        'variables': ['concept'],
        'descriptions': ["I'm having trouble understanding"],
        'placeholders': ['']
}]

In [ ]:
chatbot_model = ChatBotModel(madlibs_list)

In [ ]:
chatbot_model.selected_index = 1

In [ ]:
chatbot_model.prompt()

In [ ]:
chatbot_model.madlib_models[0].variables

In [ ]:
#|export
class ChatBotView(widgets.VBox):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    
    def __init__(self):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat = Textarea(
            disabled = True,
            layout=Layout(width='90%', height='400px')
        )
        self.radio_buttons = widgets.RadioButtons()
        self.hbox = widgets.HBox()
        self.default = widgets.Text(
            placeholder='Message AI chatbot...',
            #layout=Layout(width='100%')
        )
        self.stack = widgets.Stack()        
        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.hbox.children = (self.stack, self.submit_button)
        self.children = (self.chat, self.radio_buttons, self.hbox ) 

In [ ]:
chatbotview = ChatBotView()
chatbotview

In [ ]:
#|export
class ChatBot(ChatBotView):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    selected_index = traitlets.Int()
    
    def __init__(self, model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()
        self.model = model
       
        self.radio_buttons.options = [(name, i) for i, name in enumerate(self.model.names)]
        traitlets.link((self, 'selected_index'), (self.radio_buttons, 'value'))

        madlib_views = []
        for model in self.model.madlib_models:
            view = MadLibView(model)
            madlib_views.append(view)
            self.stack.children += (view, )

        traitlets.link((self, 'selected_index'), (self.stack, 'selected_index'))
        traitlets.link((self, 'selected_index'), (self.model, 'selected_index'))

        self.submit_button.on_click(self.on_click)

    def on_click(self, change):
        for child in self.stack.children[self.selected_index].children:
            child.value = ''
        user, bot = self.model.prompt()
        self.chat.value = "USER: " + user + '\n\n' + "CHATBOT: " + bot + '\n\n'
        self.selected_index = 0


In [ ]:
chatbot = ChatBot(chatbot_model)
chatbot

In [ ]:
chatbot.model.selected_index

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()